# L6: Build Your Own RAG Bot

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md
from unstructured.partition.pptx import partition_pptx
from unstructured.staging.base import dict_to_elements

import chromadb

INFO: NumExpr defaulting to 8 threads.


In [3]:
from Utils import Utils
utils = Utils()

DLAI_API_KEY = utils.get_dlai_api_key()
DLAI_API_URL = utils.get_dlai_url()

s = UnstructuredClient(
    api_key_auth=DLAI_API_KEY,
    server_url=DLAI_API_URL,
)

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access Utils File and Helper Functions:</b> To access helper functions and other related files for this notebook, 1) click on the <em>"View"</em> option on the top menu of the notebook and then 2) click on <em>"File Browser"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>


## Example Application: Question Answering about the Donut Model

In [ ]:
from IPython.display import Image
Image(filename='images/donut_paper.png', height=400, width=400)

In [ ]:
Image(filename='images/donut_slide.png', height=400, width=400) 

In [ ]:
Image(filename='images/donut_readme.png', height=600, width=600) 

## Preprocess the PDF

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Wait Time)</code>:</b> The following block can take a few minutes to complete.</p>

In [4]:
filename = "example_files/Fenomeni magnetici e induttori.pdf"

with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(),
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
    pdf_infer_table_structure=True,
    skip_infer_table_types=[],
)

try:
    resp = s.general.partition(req)
    pdf_elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

In [5]:
pdf_elements[0].to_dict()

{'type': 'Image',
 'element_id': 'e3b0c44298fc1c149afbf4c8996fb924',
 'text': '',
 'metadata': {'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'filename': 'Fenomeni magnetici e induttori.pdf'}}

In [6]:
tables = [el for el in pdf_elements if el.category == "Table"]

In [7]:
table_html = tables[0].metadata.text_as_html

In [8]:
from io import StringIO 
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True)
file_obj = StringIO(table_html)
tree = etree.parse(file_obj, parser)
print(etree.tostring(tree, pretty_print=True).decode())

<table>
  <tr>
    <td>471077</td>
    <td>H/m</td>
  </tr>
</table>



In [ ]:
Image(filename='images/donut_references.png', height=400, width=400) 

In [9]:
reference_title = [
    el for el in pdf_elements
    if el.text == "References"
    and el.category == "Title"
][0]

IndexError: list index out of range

In [ ]:
reference_title.to_dict()

In [ ]:
references_id = reference_title.id

In [ ]:
for element in pdf_elements:
    if element.metadata.parent_id == references_id:
        print(element)
        break

In [ ]:
pdf_elements = [el for el in pdf_elements if el.metadata.parent_id != references_id]

### Filter out headers

In [ ]:
Image(filename='images/donut_headers.png', height=400, width=400) 

In [11]:
headers = [el for el in pdf_elements if el.category == "Header"]

In [12]:
headers[1].to_dict()

{'type': 'Header',
 'element_id': 'a50eab9333a59d648039f387b1ab3d3d',
 'text': '12',
 'metadata': {'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 12,
  'filename': 'Fenomeni magnetici e induttori.pdf'}}

In [13]:
pdf_elements = [el for el in pdf_elements if el.category != "Header"]

In [14]:
pdf_elements

## Preprocess the PowerPoint Slide

In [10]:
filename = "example_files/Fenomeni magnetici e induttori.pptx"
pptx_elements = partition_pptx(filename=filename)

## Preprocess the README

In [ ]:
filename = "example_files/donut_readme.md"

In [ ]:
md_elements = partition_md(filename=filename)

## Load the Documents into the Vector DB

In [15]:
elements = chunk_by_title(pdf_elements + pptx_elements)

In [16]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [17]:
documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [18]:
embeddings = OpenAIEmbeddings()

In [21]:
vectorstore = Chroma.from_documents(documents, embeddings)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [22]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

In [23]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [24]:
template = """You are an AI assistant for answering questions about the Donut document understanding model.
You are given the following extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not about Donut, politely inform them that you are tuned to only answer questions about Donut.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [25]:
llm = OpenAI(temperature=0)

doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

In [27]:
qa_chain.invoke({
    "question": "Puoi spiegarmi la legge di Lenz e le sue applicazioni? Puoi mostrare anche la formula?",
    "chat_history": []
})["answer"]

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


' La legge di Lenz afferma che ogni variazione del flusso concatenato produce un effetto che si oppone alla propria causa, ossia che é tale da ridurre la variazione di flusso stesso. La formula della legge di Lenz è ε = -dΦ/dt. Questa legge ha numerose applicazioni pratiche, ad esempio nel funzionamento dei generatori elettrici e dei trasformatori. SOURCES: Fenomeni magnetici e induttori.pdf'

In [31]:
filter_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1, "filter": {"source": "Fenomeni magnetici e induttori.pptx"}}
)

In [32]:
filter_chain = ConversationalRetrievalChain(
    retriever=filter_retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

In [35]:
filter_chain.invoke({
    "question": "Cosa sono i materiali ferromagnetici? Quali sono le loro principali caratteristiche? Mi puoi mostrare un esempio di curva di prima magnetizzazione?",
    "chat_history": [],
    "filter": filter,
})["answer"]

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


' I materiali ferromagnetici sono materiali che presentano proprietà magnetiche molto elevate, come ad esempio il ferro, il nichel e il cobalto. Le loro principali caratteristiche sono la capacità di essere magnetizzati e di mantenere la magnetizzazione anche dopo la rimozione del campo magnetico esterno. Un esempio di curva di prima magnetizzazione è mostrato nella figura 3.2 del libro "Elettrotecnica Vol. I" di Chitarin-Gnesotto-Guarnieri-Maschio-Stella.\nSOURCES: Fenomeni magnetici e induttori.pptx'

## Work With Your Own Files

In [ ]:
import panel as pn
#import param
from Utils import upld_file
pn.extension()

upld_widget = upld_file()
pn.Row(upld_widget.widget_file_upload)

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> 🖥 &nbsp; <b>Note:</b> If the file upload interface isn't functioning properly, the issue may be related to your browser version. In such a case, please ensure your browser is updated to the latest version, or try using a different browser.</p>


In [ ]:
!ls ./example_files

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Uploading Your Own File - Method 2:</b> To upload your own files, you can also 1) click on the <em>"View"</em> option on the top menu of the notebook and then 2) click on <em>"File Browser"</em>. Then 3) click on <em>"Upload"</em> button to upload your files. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>